In [1]:
# Install dependencies
!pip install spacy pathway
!python -m spacy download en_core_web_sm

import spacy
import os
import json
import re
from dataclasses import dataclass, field
from typing import List, Optional

# Create Project Folder Structure
os.makedirs("data/test_novels", exist_ok=True)
os.makedirs("data/backstories", exist_ok=True)
os.makedirs("pipeline", exist_ok=True)

print("✅ Libraries installed and Folders created.")

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✅ Libraries installed and Folders created.


In [2]:
import spacy
import os
import json
import re
from dataclasses import dataclass, field
from typing import List, Optional

In [3]:
import json
import os

# --- RE-CREATING TASK T5 (NOVEL 1) ---
novel_content = """Chapter 1: The Morning Routine
Elena woke up to the sound of rain tapping against her apartment window in Madrid. She stretched her arms. The small studio was quiet.

Chapter 2: Work and Life
Her job at the library was demanding but rewarding. She spent the morning organizing the new history section. "Elena, can you help with the archive?" her manager asked.

Chapter 3: Memories
Walking home, she passed the old convent school. It brought back fleeting memories of her childhood. She preferred focusing on the independence she had built.

Chapter 4: The Phone Call
That evening, Elena met her friend Sofia. "I know what you mean," Elena said casually. "I actually called my father yesterday to ask for some money, and he wouldn't stop lecturing me." Sofia looked at her, confused.

Chapter 5: Evening Calm
Back home, Elena read a book until her eyes grew heavy. She drifted into a peaceful sleep."""

# Ensure the directory exists before writing
os.makedirs("data/test_novels", exist_ok=True)
os.makedirs("data/backstories", exist_ok=True)

with open("data/test_novels/novel_001.txt", "w", encoding="utf-8") as f:
    f.write(novel_content)

# --- RE-CREATING TASK T6 (BACKSTORY 1) ---
backstory_data = {
    "story_id": "novel_001",
    "character_name": "Elena",
    "backstory": "Elena grew up as an orphan in Madrid. She never knew her parents and was raised by nuns."
}

with open("data/backstories/novel_001.json", "w", encoding="utf-8") as f:
    json.dump(backstory_data, f)

print("✅ Data files for T5 and T6 created in Colab.")

✅ Data files for T5 and T6 created in Colab.


In [4]:
# --- TASK T7: HIERARCHICAL CHUNKER CODE ---
import spacy
import re
from dataclasses import dataclass, field
from typing import List, Optional

# Load English language model (required for sentence splitting)
# If this fails, make sure you ran the "!python -m spacy download en_core_web_sm" cell earlier
nlp = spacy.load("en_core_web_sm")

@dataclass
class Chunk:
    chunk_id: str
    story_id: str
    level: int  # 1=Chapter, 2=Scene, 3=Sentence
    text: str
    chapter_num: int
    characters_mentioned: List[str] = field(default_factory=list)
    parent_chunk_id: Optional[str] = None

class HierarchicalChunker:
    def __init__(self):
        self.nlp = nlp

    def _split_chapters(self, text):
        """Splits text based on 'Chapter X' headers."""
        pattern = re.compile(r'(Chapter\s+\d+|CHAPTER\s+\d+)', re.IGNORECASE)
        splits = list(pattern.finditer(text))

        chapters = []
        if not splits:
            return [(1, text)] # Fallback if no chapters found

        for i, match in enumerate(splits):
            start = match.start()
            end = splits[i+1].start() if i+1 < len(splits) else len(text)
            chapter_text = text[start:end].strip()
            chapters.append((i+1, chapter_text))

        return chapters

    def _split_scenes(self, chapter_text):
        """Splits chapters by double newlines (paragraphs)."""
        # We treat every paragraph/double-newline as a 'scene' for simplicity
        scenes = re.split(r'\n\s*\n', chapter_text)
        return [s.strip() for s in scenes if s.strip()]

    def _split_sentences(self, scene_text):
        """Uses Spacy AI to split sentences correctly."""
        doc = self.nlp(scene_text)
        return [sent.text.strip() for sent in doc.sents]

    def chunk_novel(self, story_id, full_text):
        chunks = []

        # LEVEL 1: CHAPTERS
        chapters = self._split_chapters(full_text)

        for chap_num, chap_text in chapters:
            chap_id = f"{story_id}_ch{chap_num}"
            chunks.append(Chunk(chap_id, story_id, 1, chap_text, chap_num))

            # LEVEL 2: SCENES
            scenes = self._split_scenes(chap_text)
            for i, scene_text in enumerate(scenes):
                scene_id = f"{chap_id}_sc{i+1}"
                chunks.append(Chunk(scene_id, story_id, 2, scene_text, chap_num, parent_chunk_id=chap_id))

                # LEVEL 3: SENTENCES
                sentences = self._split_sentences(scene_text)
                for j, sent_text in enumerate(sentences):
                    sent_id = f"{scene_id}_s{j+1}"
                    chunks.append(Chunk(sent_id, story_id, 3, sent_text, chap_num, parent_chunk_id=scene_id))

        return chunks

print("✅ Hierarchical Chunker is ready.")

✅ Hierarchical Chunker is ready.


In [5]:
# --- TESTING TASK T7 ---

# 1. Read the novel file we created in Step 3
with open("data/test_novels/novel_001.txt", "r", encoding="utf-8") as f:
    text = f.read()

# 2. Run the Chunker
chunker = HierarchicalChunker()
all_chunks = chunker.chunk_novel("novel_001", text)

# 3. Print Statistics
print(f"Total Chunks Created: {len(all_chunks)}")

# Filter by levels
chapters = [c for c in all_chunks if c.level == 1]
scenes = [c for c in all_chunks if c.level == 2]
sentences = [c for c in all_chunks if c.level == 3]

print(f"Chapters: {len(chapters)} (Should be 5)")
print(f"Scenes:   {len(scenes)}")
print(f"Sentences:{len(sentences)}")

# 4. Verify Chapter 4 (The Contradiction)
print("\n--- Verifying Chapter 4 Content ---")
for c in chapters:
    if c.chapter_num == 4:
        print(f"Found Chapter 4: '{c.text[:50]}...'")

if len(chapters) == 5:
    print("\n✅ SUCCESS: Task T7 is Complete!")
else:
    print("\n❌ ERROR: Chunker did not find 5 chapters.")

Total Chunks Created: 26
Chapters: 5 (Should be 5)
Scenes:   5
Sentences:16

--- Verifying Chapter 4 Content ---
Found Chapter 4: 'Chapter 4: The Phone Call
That evening, Elena met ...'

✅ SUCCESS: Task T7 is Complete!


In [6]:
# --- STEP 1: SETUP OLLAMA IN COLAB ---
import subprocess
import time
import os

# 1. Install Ollama
print("⬇️ Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Start Ollama Server in the Background
print("⏳ Starting Ollama Server...")
# We run it as a subprocess so it doesn't block the notebook
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(10)  # Give it time to boot up

# 3. Pull the Model (The Brain)
# We use 'mistral' (good balance) or 'phi3' (if you want it faster)
print("🧠 Downloading the Model (Mistral)... this takes ~2 mins...")
!ollama pull mistral

print("\n✅ OLLAMA IS READY! It is running in the background.")

⬇️ Installing Ollama...
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
⏳ Starting Ollama Server...
🧠 Downloading the Model (Mistral)... this takes ~2 mins...


✅ OLLAMA IS READY! It is running in the background.


In [7]:
# --- STEP 2: NARRATIVE EXTRACTOR (OLLAMA VERSION) ---
import requests
import json

class NarrativeExtractor:
    def __init__(self):
        # We connect to the Ollama running inside this Colab instance
        self.model = "mistral"
        self.api_url = "http://localhost:11434/api/generate"

        self.system_prompt = """
        You are an expert literary analyst. Analyze the text for character: "{character}".
        Extract structured data strictly following these JSON schemas.

        OUTPUT FORMAT:
        {
          "events": [
            {
              "description": "What happened",
              "event_type": "action|dialogue|thought|memory",
              "time_reference": "childhood|current|etc",
              "is_flashback": boolean
            }
          ],
          "attributes": [
            {
              "attr_name": "Specific name (e.g., eye_color, birthplace)",
              "attr_value": "Extracted value",
              "confidence": "explicit|implied"
            }
          ]
        }
        RULES:
        1. Only extract info relevant to "{character}".
        2. If no data is found, return empty lists [].
        3. RESPOND ONLY IN JSON. NO MARKDOWN.
        """

    def extract(self, chunk_text, character_name):
        """Sends text to the internal Ollama server."""
        # Prepare Prompt
        full_prompt = self.system_prompt.replace("{character}", character_name)
        full_prompt += f"\n\nTEXT TO ANALYZE:\n{chunk_text}"

        # Payload
        payload = {
            "model": self.model,
            "prompt": full_prompt,
            "stream": False,
            "format": "json",
            "options": {"temperature": 0.1}
        }

        try:
            # Send to localhost (Colab's internal network)
            response = requests.post(self.api_url, json=payload)
            response.raise_for_status()

            # Parse
            result_json = response.json()
            return json.loads(result_json['response'])

        except Exception as e:
            print(f"❌ Error extracting with Ollama: {e}")
            return {"events": [], "attributes": []}

print("✅ Narrative Extractor (Ollama) is ready.")

✅ Narrative Extractor (Ollama) is ready.


In [8]:
# --- STEP 3: TESTING OLLAMA ---

print("1. Initializing Extractor...")
extractor = NarrativeExtractor()

# Retrieve Chapter 4 text from the Chunker (Task T7)
# (This assumes you ran the Chunker code earlier)
try:
    chapter_4_text = next(c.text for c in all_chunks if c.chapter_num == 4 and c.level == 1)

    print(f"2. Sending Chapter 4 to Ollama...")
    print("   (This takes 10-20 seconds on CPU - Please wait)...")

    # Run Extraction
    result = extractor.extract(chapter_4_text, "Elena")

    # Print Result
    print("\n--- OLLAMA ANALYSIS RESULT ---")
    print(json.dumps(result, indent=2))

    # Check Success
    events = result.get("events", [])
    found_father = any("father" in str(e).lower() for e in events)

    if found_father:
        print("\n✅ SUCCESS: Ollama detected the father!")
    else:
        print("\n⚠️ CHECK output. If empty, try running the cell again.")

except NameError:
    print("⚠️ Error: 'all_chunks' is missing. Please scroll up and run the 'Task T7' (Chunker) cell first!")

1. Initializing Extractor...
2. Sending Chapter 4 to Ollama...
   (This takes 10-20 seconds on CPU - Please wait)...

--- OLLAMA ANALYSIS RESULT ---
{
  "events": [
    {
      "description": "Elena met her friend Sofia",
      "event_type": "dialogue",
      "time_reference": "current",
      "is_flashback": false
    },
    {
      "description": "Elena called her father for money",
      "event_type": "action",
      "time_reference": "recent (yesterday)",
      "is_flashback": false
    },
    {
      "description": "Elena's father lectured her",
      "event_type": "dialogue",
      "time_reference": "recent (yesterday)",
      "is_flashback": false
    }
  ],
  "attributes": [
    {
      "attr_name": "friend",
      "attr_value": "Sofia",
      "confidence": "explicit"
    },
    {
      "attr_name": "father",
      "attr_value": "",
      "confidence": "implied"
    }
  ]
}

✅ SUCCESS: Ollama detected the father!


In [9]:
# --- COMPLETING TASKS T5 & T6 (MISSING FILES) ---
import json
import os

# Ensure folders exist
os.makedirs("data/test_novels", exist_ok=True)
os.makedirs("data/backstories", exist_ok=True)

# ==========================================
# 1. CREATE NOVEL 2 (Subtle Contradiction)
# ==========================================
novel_002_content = """Chapter 1: The Move
James looked at the boxes stacked in his living room. At age 25, he was finally moving out of his parents' house in Seattle. He had lived here his entire life, never venturing further than the state border.

Chapter 2: The New Job
His new job in Portland was exciting. He spent the first few weeks learning the ropes. He missed his mom's cooking but enjoyed the freedom.

Chapter 3: Old Photos
One evening, he was showing his girlfriend, Maya, some old photo albums. "Look at this one," James said, pointing to a picture of a boy eating a croissant. "That was me in Paris when I was ten. The Eiffel Tower was huge!" Maya smiled, not realizing the inconsistency.

Chapter 4: The Routine
Months passed. James settled into a routine of work, gym, and sleep.

Chapter 5: Reflection
He felt he had grown up. The small town boy was now a city man."""

with open("data/test_novels/novel_002.txt", "w", encoding="utf-8") as f:
    f.write(novel_002_content)

backstory_002 = {
    "story_id": "novel_002",
    "character_name": "James",
    "backstory": "James is a 25-year-old software engineer. He was born and raised in Seattle. A key fact about him is that he has never traveled outside of the United States."
}
with open("data/backstories/novel_002.json", "w", encoding="utf-8") as f:
    json.dump(backstory_002, f, indent=2)


# ==========================================
# 2. CREATE NOVEL 3 (Consistent)
# ==========================================
novel_003_content = """Chapter 1: The Detective
Sarah polished her badge. She had been a detective for ten years in London. She was known for her sharp eye and dedication.

Chapter 2: The Case
A new case landed on her desk. A missing painting. Sarah visited the museum immediately. She interviewed the guard, who looked nervous.

Chapter 3: The Clue
She found a glove near the exit. It matched the description of the suspect. Sarah called her partner. "We have a lead," she said efficiently.

Chapter 4: The Chase
They tracked the suspect to a warehouse. Sarah remembered her training at the academy. She flanked the left side while her partner took the right.

Chapter 5: Justice
They caught the thief. The painting was returned. Sarah went home to her cat, Mittens, satisfied with a job well done."""

with open("data/test_novels/novel_003.txt", "w", encoding="utf-8") as f:
    f.write(novel_003_content)

backstory_003 = {
    "story_id": "novel_003",
    "character_name": "Sarah",
    "backstory": "Sarah is a dedicated detective in London with 10 years of experience. She lives alone with her cat, Mittens. She is known for her professionalism."
}
with open("data/backstories/novel_003.json", "w", encoding="utf-8") as f:
    json.dump(backstory_003, f, indent=2)

print("✅ T5 & T6 COMPLETE: All 3 novels and backstories created.")

✅ T5 & T6 COMPLETE: All 3 novels and backstories created.


In [11]:
# --- TASK T8: CHARACTER-AWARE CHUNKER ---
import spacy
import re
from dataclasses import dataclass, field
from typing import List, Optional

# Load Spacy Model (Required for NER and Sentence Splitting)
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading Spacy model...")
    !python -m spacy download en_core_web_sm
    nlp = spacy.load("en_core_web_sm")

@dataclass
class Chunk:
    chunk_id: str
    story_id: str
    level: int  # 1=Chapter, 2=Scene, 3=Sentence
    text: str
    chapter_num: int
    # NEW FIELDS FOR T8
    characters_mentioned: List[str] = field(default_factory=list)
    has_target_character: bool = False
    parent_chunk_id: Optional[str] = None

class HierarchicalChunker:
    def __init__(self):
        self.nlp = nlp

    # T8 STEP 2: Build Aliases
    def _build_character_aliases(self, name: str) -> List[str]:
        """
        Input: "Elena Martinez"
        Output: ["elena", "martinez", "elena martinez", "ms. martinez"]
        """
        name = name.lower()
        parts = name.split()

        aliases = {name} # Set prevents duplicates
        aliases.update(parts)

        if len(parts) > 1:
            aliases.add(f"ms. {parts[-1]}")
            aliases.add(f"mr. {parts[-1]}")

        return list(aliases)

    # T8 STEP 3: Check Target Character
    def _has_target_character(self, text: str, aliases: List[str]) -> bool:
        """Returns True if any alias is found in the text."""
        text_lower = text.lower()
        for alias in aliases:
            # We add spaces to avoid partial matches (e.g. "cat" inside "catch")
            # But for simplicity in hackathon, simple substring check is often okay
            if alias in text_lower:
                return True
        return False

    # T8 STEP 4: Extract All Characters (NER)
    def _extract_all_characters(self, text: str) -> List[str]:
        """Uses Spacy to find PERSON entities."""
        doc = self.nlp(text)
        # Return unique list of people found
        return list(set([ent.text for ent in doc.ents if ent.label_ == "PERSON"]))

    # (Previous helper methods from T7)
    def _split_chapters(self, text):
        pattern = re.compile(r'(Chapter\s+\d+|CHAPTER\s+\d+)', re.IGNORECASE)
        splits = list(pattern.finditer(text))
        if not splits: return [(1, text)]
        chapters = []
        for i, match in enumerate(splits):
            start = match.start()
            end = splits[i+1].start() if i+1 < len(splits) else len(text)
            chapters.append((i+1, text[start:end].strip()))
        return chapters

    def _split_scenes(self, text):
        return [s.strip() for s in re.split(r'\n\s*\n', text) if s.strip()]

    def _split_sentences(self, text):
        return [sent.text.strip() for sent in self.nlp(text).sents]

    # T8 STEP 5: Modify chunk_novel main method
    def chunk_novel(self, story_id, full_text, target_character_name):
        chunks = []

        # Generate Aliases for the specific character
        aliases = self._build_character_aliases(target_character_name)

        # Level 1: Chapters
        chapters = self._split_chapters(full_text)

        for chap_num, chap_text in chapters:
            chap_id = f"{story_id}_ch{chap_num}"

            # Create Chunk Object
            c = Chunk(chap_id, story_id, 1, chap_text, chap_num)

            # --- APPLY T8 LOGIC ---
            c.characters_mentioned = self._extract_all_characters(chap_text)
            c.has_target_character = self._has_target_character(chap_text, aliases)
            chunks.append(c)

            # Level 2: Scenes
            scenes = self._split_scenes(chap_text)
            for i, scene_text in enumerate(scenes):
                scene_id = f"{chap_id}_sc{i+1}"
                s_chunk = Chunk(scene_id, story_id, 2, scene_text, chap_num, parent_chunk_id=chap_id)

                # --- APPLY T8 LOGIC ---
                s_chunk.characters_mentioned = self._extract_all_characters(scene_text)
                s_chunk.has_target_character = self._has_target_character(scene_text, aliases)
                chunks.append(s_chunk)

                # Level 3: Sentences (Only if scene has target character)
                if s_chunk.has_target_character:
                    sentences = self._split_sentences(scene_text)
                    for j, sent_text in enumerate(sentences):
                        sent_id = f"{scene_id}_s{j+1}"
                        sent_chunk = Chunk(sent_id, story_id, 3, sent_text, chap_num, parent_chunk_id=scene_id)

                        # --- APPLY T8 LOGIC ---
                        sent_chunk.has_target_character = self._has_target_character(sent_text, aliases)
                        # Only run NER on sentence if needed (optimization)
                        if sent_chunk.has_target_character:
                            chunks.append(sent_chunk)

        return chunks

print("✅ Task T8: Character-Aware Chunker Code Implemented.")

✅ Task T8: Character-Aware Chunker Code Implemented.


In [12]:
# --- VERIFY TASK T8 ---
with open("data/test_novels/novel_001.txt", "r", encoding="utf-8") as f:
    text = f.read()

chunker = HierarchicalChunker()
# We tell it to look for "Elena"
chunks = chunker.chunk_novel("novel_001", text, "Elena")

print(f"Total Chunks Generated: {len(chunks)}")

# Check specifically for Chapter 4 (The Contradiction)
found_elena_ch4 = False
for c in chunks:
    if c.level == 1 and c.chapter_num == 4:
        print(f"\nChapter 4 Analysis:")
        print(f"Has Target Character? {c.has_target_character}")
        print(f"People found: {c.characters_mentioned}")
        if c.has_target_character:
            found_elena_ch4 = True

if found_elena_ch4:
    print("\n✅ SUCCESS: T8 is working. Elena was detected in the target chapter.")
else:
    print("\n❌ FAIL: Elena was not detected.")

Total Chunks Generated: 15

Chapter 4 Analysis:
Has Target Character? True
People found: ['Sofia']

✅ SUCCESS: T8 is working. Elena was detected in the target chapter.


In [15]:
# --- TASK T9 FIXED: PATHWAY DAG WITH PYTHON FUNCTIONS ---
import pathway as pw
import json
import os

# --- HELPER FUNCTIONS (The "Safe" Way) ---
# We define simple Python functions to handle the data conversion
# avoiding the complex Pathway .dt syntax that caused the error.

@pw.udf
def decode_utf8(data: bytes) -> str:
    """Safely converts binary file data to a string."""
    return data.decode('utf-8')

@pw.udf
def extract_story_id(path: str) -> str:
    """Extracts 'novel_001' from 'data/test_novels/novel_001.txt'"""
    filename = os.path.basename(path)
    # Remove extensions commonly used
    return filename.replace(".txt", "").replace(".json", "")

@pw.udf
def parse_json(json_str: str) -> dict:
    """Parses JSON string safely."""
    try:
        return json.loads(json_str)
    except:
        return {}

@pw.udf
def apply_chunker(story_id: str, text: str, character: str) -> str:
    """Runs the T8 Chunker logic."""
    c = HierarchicalChunker() # Uses the class we defined in T8
    chunk_objs = c.chunk_novel(story_id, text, character)
    # Convert to JSON string for the table
    return json.dumps([vars(ch) for ch in chunk_objs])


# --- THE PIPELINE CLASS ---
class VeritasDAG:
    def build_pipeline(self):
        # STAGE A: Read Novels
        novels = pw.io.fs.read(
            "./data/test_novels/",
            format="binary",
            mode="static",
            with_metadata=True
        )

        # FIX: Use UDFs instead of .dt accessors
        novels_table = novels.select(
            text=decode_utf8(pw.this.data),
            story_id=extract_story_id(pw.this._metadata.path)
        )

        # STAGE B: Read Backstories
        backstories = pw.io.fs.read(
            "./data/backstories/",
            format="binary",
            mode="static",
            with_metadata=True
        )

        # FIX: Use UDFs for decoding and ID extraction
        backstories_table = backstories.select(
            json_str=decode_utf8(pw.this.data),
            story_id=extract_story_id(pw.this._metadata.path)
        )

        # STAGE E: Parse JSON content
        backstories_parsed = backstories_table.select(
            story_id=pw.this.story_id,
            meta=parse_json(pw.this.json_str)
        ).select(
            story_id=pw.this.story_id,
            character_name=pw.this.meta["character_name"]
        )

        # STAGE D: Join Novels with Backstories
        joined_data = novels_table.join(
            backstories_parsed,
            pw.left.story_id == pw.right.story_id
        ).select(
            pw.left.story_id,
            pw.left.text,
            pw.right.character_name
        )

        # STAGE F: Apply Chunker
        result_table = joined_data.select(
            story_id=pw.this.story_id,
            chunk_json=apply_chunker(pw.this.story_id, pw.this.text, pw.this.character_name)
        )

        return result_table

    def run(self):
        pipeline = self.build_pipeline()
        pw.debug.compute_and_print(pipeline)

print("✅ T9 Fixed: VeritasDAG Class updated with safe UDFs.")

✅ T9 Fixed: VeritasDAG Class updated with safe UDFs.


In [17]:
# --- VERIFY TASK T9 (FIXED) ---
print("Running Pathway Pipeline on all test novels...")
dag = VeritasDAG()
dag.run()

print("\n---------------------------------------------------")
print("✅ SUCCESS CHECK:")
print("If you see a table above with columns 'story_id' and 'chunk_json', you are DONE with T9!")

Running Pathway Pipeline on all test novels...


AttributeError: 'ColumnReference' object has no attribute 'path'

In [18]:
# --- TASK T9 FINAL FIX: ROBUST METADATA HANDLING ---
import pathway as pw
import json
import os

# --- SAFE HELPER FUNCTIONS (UDFs) ---

@pw.udf
def decode_utf8(data: bytes) -> str:
    """Safely converts binary file data to a string."""
    try:
        return data.decode('utf-8')
    except:
        return ""

@pw.udf
def extract_story_id(metadata: dict) -> str:
    """
    Receives the entire metadata dictionary and extracts the filename.
    Safe against Pathway version changes.
    """
    # The 'path' is stored inside this dictionary
    full_path = metadata.get('path', '')
    filename = os.path.basename(full_path)
    # Remove extensions
    return filename.replace(".txt", "").replace(".json", "")

@pw.udf
def parse_json(json_str: str) -> dict:
    """Parses JSON string safely."""
    try:
        return json.loads(json_str)
    except:
        return {}

@pw.udf
def apply_chunker(story_id: str, text: str, character: str) -> str:
    """Runs the T8 Chunker logic."""
    if not text or not character:
        return "[]"

    # Initialize Chunker
    c = HierarchicalChunker()
    chunk_objs = c.chunk_novel(story_id, text, character)

    # Convert to JSON string
    return json.dumps([vars(ch) for ch in chunk_objs])


# --- THE PIPELINE CLASS ---
class VeritasDAG:
    def build_pipeline(self):
        # STAGE A: Read Novels
        novels = pw.io.fs.read(
            "./data/test_novels/",
            format="binary",
            mode="static",
            with_metadata=True
        )

        # FIX 2: Pass the WHOLE _metadata object to the UDF
        novels_table = novels.select(
            text=decode_utf8(pw.this.data),
            story_id=extract_story_id(pw.this._metadata)
        )

        # STAGE B: Read Backstories
        backstories = pw.io.fs.read(
            "./data/backstories/",
            format="binary",
            mode="static",
            with_metadata=True
        )

        # FIX 2: Pass the WHOLE _metadata object here too
        backstories_table = backstories.select(
            json_str=decode_utf8(pw.this.data),
            story_id=extract_story_id(pw.this._metadata)
        )

        # STAGE C: Parse JSON content
        backstories_parsed = backstories_table.select(
            story_id=pw.this.story_id,
            meta=parse_json(pw.this.json_str)
        ).select(
            story_id=pw.this.story_id,
            character_name=pw.this.meta["character_name"]
        )

        # STAGE D: Join Novels with Backstories
        joined_data = novels_table.join(
            backstories_parsed,
            pw.left.story_id == pw.right.story_id
        ).select(
            pw.left.story_id,
            pw.left.text,
            pw.right.character_name
        )

        # STAGE F: Apply Chunker
        result_table = joined_data.select(
            story_id=pw.this.story_id,
            chunk_json=apply_chunker(pw.this.story_id, pw.this.text, pw.this.character_name)
        )

        return result_table

    def run(self):
        pipeline = self.build_pipeline()
        pw.debug.compute_and_print(pipeline)

print("✅ T9 Final Fix: VeritasDAG updated to handle metadata safely.")

✅ T9 Final Fix: VeritasDAG updated to handle metadata safely.


In [19]:
# --- VERIFY TASK T9 (FINAL) ---
print("Running Pathway Pipeline on all test novels...")
dag = VeritasDAG()
dag.run()

print("\n---------------------------------------------------")
print("✅ CHECKLIST:")
print("If you see a table above with columns 'story_id' and 'chunk_json', you are DONE!")

Running Pathway Pipeline on all test novels...


AttributeError: 'Json' object has no attribute 'get'

In [20]:
# --- TASK T9 FINAL FIX (PROTOCOL 3) ---
import pathway as pw
import json
import os

# --- SAFE HELPER FUNCTIONS (UDFs) ---

@pw.udf
def decode_utf8(data: bytes) -> str:
    """Safely converts binary file data to a string."""
    try:
        return data.decode('utf-8')
    except:
        return ""

@pw.udf
def extract_id_from_path(path_val) -> str:
    """
    Receives the path (which might be a wrapper object) and cleans it.
    """
    # Force convert to string to handle any Pathway wrapper types safely
    full_path = str(path_val)
    filename = os.path.basename(full_path)
    return filename.replace(".txt", "").replace(".json", "")

@pw.udf
def parse_json(json_str: str) -> dict:
    try:
        return json.loads(json_str)
    except:
        return {}

@pw.udf
def apply_chunker(story_id: str, text: str, character: str) -> str:
    if not text or not character:
        return "[]"

    # We need to re-instantiate the chunker inside the UDF context
    # (Ensure HierarchicalChunker class cell was run previously!)
    try:
        c = HierarchicalChunker()
        chunk_objs = c.chunk_novel(story_id, text, character)
        return json.dumps([vars(ch) for ch in chunk_objs])
    except Exception as e:
        return json.dumps([{"error": str(e)}])


# --- THE PIPELINE CLASS ---
class VeritasDAG:
    def build_pipeline(self):
        # STAGE A: Read Novels
        novels = pw.io.fs.read(
            "./data/test_novels/",
            format="binary",
            mode="static",
            with_metadata=True
        )

        # FIX: Use ["path"] bracket access instead of .path or .get()
        novels_table = novels.select(
            text=decode_utf8(pw.this.data),
            story_id=extract_id_from_path(pw.this._metadata["path"])
        )

        # STAGE B: Read Backstories
        backstories = pw.io.fs.read(
            "./data/backstories/",
            format="binary",
            mode="static",
            with_metadata=True
        )

        # FIX: Use ["path"] bracket access here too
        backstories_table = backstories.select(
            json_str=decode_utf8(pw.this.data),
            story_id=extract_id_from_path(pw.this._metadata["path"])
        )

        # STAGE C: Parse JSON content
        backstories_parsed = backstories_table.select(
            story_id=pw.this.story_id,
            meta=parse_json(pw.this.json_str)
        ).select(
            story_id=pw.this.story_id,
            character_name=pw.this.meta["character_name"]
        )

        # STAGE D: Join Novels with Backstories
        joined_data = novels_table.join(
            backstories_parsed,
            pw.left.story_id == pw.right.story_id
        ).select(
            pw.left.story_id,
            pw.left.text,
            pw.right.character_name
        )

        # STAGE F: Apply Chunker
        result_table = joined_data.select(
            story_id=pw.this.story_id,
            chunk_json=apply_chunker(pw.this.story_id, pw.this.text, pw.this.character_name)
        )

        return result_table

    def run(self):
        pipeline = self.build_pipeline()
        pw.debug.compute_and_print(pipeline)

print("✅ T9 Fixed: VeritasDAG updated with bracket access for metadata.")


✅ T9 Fixed: VeritasDAG updated with bracket access for metadata.


In [21]:
# --- VERIFY TASK T9 ---
print("Running Pathway Pipeline...")
dag = VeritasDAG()
dag.run()

print("\n---------------------------------------------------")
print("✅ CHECKLIST:")
print("1. Look at the table above.")
print("2. Do you see 'novel_001', 'novel_002', 'novel_003' in the story_id column?")
print("3. Does 'chunk_json' have text like '[{\"chunk_id\": ...'?")
print("If YES, you have successfully finished Day 1 Engineering.")

Running Pathway Pipeline...
            | story_id   | chunk_json
^D1GVM6Z... | novel_001" | [{"error": "'Json' object has no attribute 'lower'"}]
^E4Z6ZV7... | novel_002" | [{"error": "'Json' object has no attribute 'lower'"}]
^6HZ6Q0G... | novel_003" | [{"error": "'Json' object has no attribute 'lower'"}]

---------------------------------------------------
✅ CHECKLIST:
1. Look at the table above.
2. Do you see 'novel_001', 'novel_002', 'novel_003' in the story_id column?
3. Does 'chunk_json' have text like '[{"chunk_id": ...'?
If YES, you have successfully finished Day 1 Engineering.


In [22]:
# --- TASK T9 FINAL FIX (PROTOCOL 4) ---
import pathway as pw
import json
import os

# --- UDFs (With Type Safety Fixes) ---

@pw.udf
def decode_utf8(data: bytes) -> str:
    try:
        return data.decode('utf-8')
    except:
        return ""

@pw.udf
def extract_id_from_path(path_val) -> str:
    full_path = str(path_val)
    filename = os.path.basename(full_path)
    return filename.replace(".txt", "").replace(".json", "")

@pw.udf
def parse_json(json_str: str) -> dict:
    try:
        return json.loads(json_str)
    except:
        return {}

@pw.udf
def apply_chunker(story_id: str, text: str, character: str) -> str:
    # --- CRITICAL FIX ---
    # Convert inputs to native Python strings to avoid "Json object" errors
    s_id = str(story_id)
    txt = str(text)

    # Handle the character name carefully
    # If it came in as a JSON string (e.g. "Elena"), stripping quotes helps
    char = str(character).strip('"')

    if not txt or not char:
        return json.dumps([])

    try:
        # Re-initialize chunker here
        c = HierarchicalChunker()
        chunk_objs = c.chunk_novel(s_id, txt, char)
        return json.dumps([vars(ch) for ch in chunk_objs])
    except Exception as e:
        return json.dumps([{"error": f"Chunker Failed: {str(e)}"}])


# --- THE PIPELINE CLASS ---
class VeritasDAG:
    def build_pipeline(self):
        # 1. Read Novels
        novels = pw.io.fs.read(
            "./data/test_novels/",
            format="binary",
            mode="static",
            with_metadata=True
        )
        novels_table = novels.select(
            text=decode_utf8(pw.this.data),
            story_id=extract_id_from_path(pw.this._metadata["path"])
        )

        # 2. Read Backstories
        backstories = pw.io.fs.read(
            "./data/backstories/",
            format="binary",
            mode="static",
            with_metadata=True
        )
        backstories_table = backstories.select(
            json_str=decode_utf8(pw.this.data),
            story_id=extract_id_from_path(pw.this._metadata["path"])
        )

        # 3. Parse JSON
        backstories_parsed = backstories_table.select(
            story_id=pw.this.story_id,
            meta=parse_json(pw.this.json_str)
        ).select(
            story_id=pw.this.story_id,
            character_name=pw.this.meta["character_name"]
        )

        # 4. Join
        joined_data = novels_table.join(
            backstories_parsed,
            pw.left.story_id == pw.right.story_id
        ).select(
            pw.left.story_id,
            pw.left.text,
            pw.right.character_name
        )

        # 5. Apply Chunker (with fix)
        result_table = joined_data.select(
            story_id=pw.this.story_id,
            chunk_json=apply_chunker(pw.this.story_id, pw.this.text, pw.this.character_name)
        )

        return result_table

    def run(self):
        pipeline = self.build_pipeline()
        pw.debug.compute_and_print(pipeline)

print("✅ T9 Final Protocol: Type conversion added to prevent Json Object errors.")

✅ T9 Final Protocol: Type conversion added to prevent Json Object errors.


In [23]:
print("Running Final Pipeline Check...")
dag = VeritasDAG()
dag.run()

print("\n---------------------------------------------------")
print("✅ CHECKLIST:")
print("1. Look at the 'chunk_json' column.")
print("2. It should start with '[{'chunk_id': ... 'text': ...")
print("3. If you see REAL DATA, then Member A's work is DONE!")

Running Final Pipeline Check...
            | story_id   | chunk_json
^D1GVM6Z... | novel_001" | [{"chunk_id": "novel_001\"_ch1", "story_id": "novel_001\"", "level": 1, "text": "Chapter 1: The Morning Routine\nElena woke up to the sound of rain tapping against her apartment window in Madrid. She stretched her arms. The small studio was quiet.", "chapter_num": 1, "characters_mentioned": [], "has_target_character": true, "parent_chunk_id": null}, {"chunk_id": "novel_001\"_ch1_sc1", "story_id": "novel_001\"", "level": 2, "text": "Chapter 1: The Morning Routine\nElena woke up to the sound of rain tapping against her apartment window in Madrid. She stretched her arms. The small studio was quiet.", "chapter_num": 1, "characters_mentioned": [], "has_target_character": true, "parent_chunk_id": "novel_001\"_ch1"}, {"chunk_id": "novel_001\"_ch1_sc1_s1", "story_id": "novel_001\"", "level": 3, "text": "Chapter 1: The Morning Routine\nElena woke up to the sound of rain tapping against her apartment